<a href="https://colab.research.google.com/github/hoad211/ClassifyingNumber/blob/main/FinalProject364HoaNam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Flatten, Dense, Dropout

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [1]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [3]:
import io 
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

#Setting Seed for future reproduction

In [4]:
np.random.seed(1)

# Understanding the dataframes

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [7]:
X_features = train.drop(labels= ["label"],axis = 1)
Y_features = train["label"]


In [8]:
# Splitting the original set into a training set and a validation set
(X_train,X_val,Y_train,Y_val) = train_test_split(X_features,Y_features,test_size=0.3, random_state = 10)
print(Y_val.shape)

(12600,)


In [9]:
X_train = X_train.values.reshape(29400, 28, 28, 1) # Adding a color channel
X_val = X_val.values.reshape(12600, 28, 28, 1) # Adding a color channel
Y_train = Y_train.values
Y_val = Y_val.values

In [ ]:
plt.imshow(X_val[0].reshape(28,28),cmap = plt.cm.binary)
plt.show()

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(10,5))
axes = axes.flatten()
idx = np.random.randint(0,29400,size=5)
for i in range(5):
    axes[i].imshow(X_train[idx[i]].reshape(28,28), cmap="gray")
    axes[i].axis('off') # hide the axes ticks
    axes[i].set_title(str(int(Y_train[idx[i]])), color= 'black', fontsize=25)
plt.show()

We apply gray scale normlization to avoid having too noises causing the machine to falsely predict the number

In [12]:
X_train = X_train / X_train.max() #255 
X_val = X_val / X_val.max()

In [ ]:
plt.imshow(X_train[0].reshape(28,28),cmap = plt.cm.binary)
plt.show()

print(X_train.max())

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(10,5))
axes = axes.flatten()
for i in range(5):
    axes[i].imshow(X_train[idx[i]].reshape(28,28), cmap='gray')
    axes[i].axis('off') # hide the axes ticks
    axes[i].set_title(str(int(Y_train[idx[i]])), color= 'black', fontsize=25)
plt.show()

In [15]:
#One hot encoding
Y_cat_train = to_categorical(Y_train, 10) # short for y_categorical_train
Y_cat_val  = to_categorical(Y_val, 10)


# CNN Model

In [16]:
# # AlexNet model
# model = Sequential()

# model.add(Conv2D(filters = 96, kernel_size = (3,3), input_shape = (28,28,1), activation = 'relu'))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(BatchNormalization())

# model.add(Conv2D(filters=384, kernel_size=(3,3), activation='relu', padding="same"))
# model.add(BatchNormalization())

# model.add(Conv2D(filters=384, kernel_size=(3,3), activation='relu', padding="same"))
# model.add(BatchNormalization())

# model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding="same"))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(BatchNormalization())

# model.add(Flatten())

# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))

# early_stopping = EarlyStopping(min_delta = 0.001,patience = 20,restore_best_weights = True,verbose = 0)

# # Compile
# model.compile(optimizer = "adam" , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

# model.summary()
# # Train
# hist = model.fit(X_train, Y_train, batch_size = 256, epochs = 50,callbacks = [early_stopping], validation_data=(X_val, Y_val))


In [17]:
#Basic CNN Structure Model
model = Sequential()

#Convolutional Layer
model.add(Conv2D(filters=64,kernel_size=(3,3), input_shape=(28,28,1),activation='relu'))# Up the number of filter if dealing with complex img
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))# Up the number of filter if dealing with complex img
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu'))# Up the number of filter if dealing with complex img
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
# 2D to 1D
model.add(Flatten())
# Dense Layer
model.add(Dense(512,activation='relu'))
# Output Layer
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [18]:
ES = EarlyStopping(monitor = "val_loss", patience = 7, restore_best_weights=True)

In [19]:
LRR = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
model.summary()

In [21]:
#diversify the data set
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
model.fit(X_train, Y_cat_train, epochs = 50, batch_size = 128128, validation_data = (X_val, Y_cat_val), verbose = 2,callbacks = [ES,LRR])

Epoch 1/50


In [ ]:
metrics = pd.DataFrame(model.history.history)
print(metrics)
metrics[["loss","val_loss"]].plot()

In [ ]:
metrics[["accuracy", "val_accuracy"]].plot()

In [ ]:
# Have the model predict the validation dataset
prediction = model.predict(X_val.reshape(12600,28,28,1))
Y_pred = np.argmax(prediction, axis = 1)

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(10,5))
axes = axes.flatten()
idx = np.random.randint(0,12600,size=5)
for i in range(5):
    axes[i].imshow(X_val[idx[i]].reshape(28,28), cmap="gray")
    axes[i].axis('off') # hide the axes ticks
    axes[i].set_title(str(int(Y_pred[idx[i]])), color= 'black', fontsize=25)
plt.show()# inspect a random 5 numbers in the test dataset predicted by the model


# Errors

In [ ]:
Y_val

In [ ]:
errors = (Y_pred != Y_val)
Y_pred_errors = Y_pred[errors]
Y_val_pred_errors = prediction[errors]
Y_val_true = Y_val[errors]
X_val_errors = X_val[errors]

In [ ]:
#Sorting the criticality of the errors
Y_pred_errors_prob = np.max(Y_val_pred_errors, axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_val_pred_errors, Y_val_true, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 5 errors 
most_important_errors = sorted_dela_errors[-5:]


In [ ]:
# display errors
fig, axes = plt.subplots(1,5,figsize=(10,5))
axes = axes.flatten()
for i in range(5):
    idx = most_important_errors[i]
    axes[i].imshow(X_val_errors[idx].reshape(28,28), cmap="gray")
    axes[i].axis('off') # hide the axes ticks
    axes[i].set_title("Predicted label :{}\nTrue label :{}".format(Y_pred_errors[idx],Y_val_true[idx]), color= 'black')
plt.show()


From the given results, there exists overfitting problem in our module
Resolution: Introduce early stopping and increase number proxies